In [ ]:
# [Previous imports remain the same]
import re
from google.colab import auth, drive
import google.auth
import gspread
import pandas as pd
import json
import os

# [Previous functions remain the same - get_answers_dict and parse_questions]

try:
    # Mount Google Drive
    print("Mounting Google Drive...")
    drive.mount('/content/drive')

    # Create directory if it doesn't exist
    target_dir = '/content/drive/MyDrive/TFM2/TFM-DATASETS'
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
        print(f"Created directory: {target_dir}")

    # [Previous authentication and data processing code remains the same until saving files]

    # Save files both locally and to Drive
    timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")

    # Save CSV
    print("Saving CSV files...")
    local_csv = f'structured_questions_{timestamp}.csv'
    drive_csv = f'{target_dir}/structured_questions_{timestamp}.csv'
    df = pd.DataFrame(structured_questions)
    df.to_csv(local_csv, index=False)
    df.to_csv(drive_csv, index=False)

    # Save JSON
    print("Saving JSON files...")
    local_json = f'structured_questions_{timestamp}.json'
    drive_json = f'{target_dir}/structured_questions_{timestamp}.json'
    with open(local_json, 'w', encoding='utf-8') as f:
        json.dump(structured_questions, f, ensure_ascii=False, indent=2)
    with open(drive_json, 'w', encoding='utf-8') as f:
        json.dump(structured_questions, f, ensure_ascii=False, indent=2)

    print(f"\nFiles saved to Google Drive at {target_dir}:")
    print(f"1. structured_questions_{timestamp}.csv")
    print(f"2. structured_questions_{timestamp}.json")

    # Verify files were saved
    if os.path.exists(drive_csv) and os.path.exists(drive_json):
        print("\nFiles successfully saved to Google Drive!")

        # Print file sizes to verify content
        csv_size = os.path.getsize(drive_csv) / 1024  # Size in KB
        json_size = os.path.getsize(drive_json) / 1024  # Size in KB
        print(f"CSV file size: {csv_size:.2f} KB")
        print(f"JSON file size: {json_size:.2f} KB")
    else:
        print("\nWarning: Some files may not have been saved correctly!")

except Exception as e:
    print(f"Error: {str(e)}")

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saving CSV files...
Saving JSON files...

Files saved to Google Drive at /content/drive/MyDrive/TFM2/TFM-DATASETS:
1. structured_questions_20250111_225818.csv
2. structured_questions_20250111_225818.json

Files successfully saved to Google Drive!
CSV file size: 97.08 KB
JSON file size: 120.54 KB


In [ ]:
import re
from google.colab import auth
import google.auth
import gspread
import pandas as pd
import json

def get_answers_dict(worksheet):
    """Get answers from the Answers Matrix worksheet"""
    answers_data = worksheet.get_all_values()
    # Skip header if exists
    if answers_data and answers_data[0][0].lower().strip() in ['question', 'questionid', 'id']:
        answers_data = answers_data[1:]

    # Create dictionary of question_number: answer
    answers_dict = {}
    for row in answers_data:
        if len(row) >= 2:  # Ensure row has both question number and answer
            question_num = row[0].strip()
            answer = row[1].strip()
            # Convert numeric answer to letter (1->A, 2->B, etc)
            if answer in ['1', '2', '3', '4']:
                answer = chr(64 + int(answer))  # 65 is ASCII for 'A'
            answers_dict[question_num] = answer

    return answers_dict

def parse_questions(raw_data, answers_dict):
    questions = []
    current_question = None
    current_options = []

    # Skip header row
    for row in raw_data[1:]:
        text = row[0].strip()

        # Check if this is a question (starts with number followed by dot)
        question_match = re.match(r'^(\d{2,3})\.\s*(.*)', text)  # Matches 26. or 210.
        option_match = re.match(r'^([1-4])\.\s*(.*)', text)      # Matches 1. to 4.

        if question_match:
            # Save previous question if exists
            if current_question and len(current_options) == 4:
                question_id = current_question['id']
                questions.append({
                    'id': question_id,
                    'question': current_question['text'],
                    'option_a': current_options[0],
                    'option_b': current_options[1],
                    'option_c': current_options[2],
                    'option_d': current_options[3],
                    'correct_answer': answers_dict.get(question_id, '')
                })

            # Start new question
            current_question = {
                'id': question_match.group(1),
                'text': question_match.group(2)
            }
            current_options = []

        elif option_match and current_question:
            current_options.append(option_match.group(2))

    # Don't forget to add the last question
    if current_question and len(current_options) == 4:
        question_id = current_question['id']
        questions.append({
            'id': question_id,
            'question': current_question['text'],
            'option_a': current_options[0],
            'option_b': current_options[1],
            'option_c': current_options[2],
            'option_d': current_options[3],
            'correct_answer': answers_dict.get(question_id, '')
        })

    return questions

try:
    # Authenticate
    print("Authenticating...")
    auth.authenticate_user()
    creds, _ = google.auth.default()
    gc = gspread.authorize(creds)

    # Your Google Sheet URL
    sheet_url = 'https://docs.google.com/spreadsheets/d/1rpqI5GlJUBmoa6gtrksgH9KrEaBXpD5xzCSgd_BH2Fw/edit?usp=sharing'

    # Open spreadsheet and get raw data
    print("Getting data from sheets...")
    spreadsheet = gc.open_by_url(sheet_url)

    # Get questions
    questions_worksheet = spreadsheet.worksheet('Questions2024')
    raw_data = questions_worksheet.get_all_values()

    # Get answers
    print("Getting answers from Answers Matrix...")
    answers_worksheet = spreadsheet.worksheet('Answers Matrix')
    answers_dict = get_answers_dict(answers_worksheet)

    # Parse questions
    print("Parsing questions and integrating answers...")
    structured_questions = parse_questions(raw_data, answers_dict)

    print(f"Successfully parsed {len(structured_questions)} questions")

    # Save as CSV
    print("Saving to CSV...")
    df = pd.DataFrame(structured_questions)
    df.to_csv('structured_questions.csv', index=False)

    # Save as JSON
    print("Saving to JSON...")
    with open('structured_questions.json', 'w', encoding='utf-8') as f:
        json.dump(structured_questions, f, ensure_ascii=False, indent=2)

    # Create new worksheet in Google Sheet
    print("Creating new worksheet...")
    try:
        new_worksheet = spreadsheet.add_worksheet('Structured_Questions', 1000, 7)
    except:
        new_worksheet = spreadsheet.worksheet('Structured_Questions')
        new_worksheet.clear()

    # Add headers
    headers = ['QuestionID', 'Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Correct Answer']
    new_worksheet.update('A1:G1', [headers])

    # Prepare data for worksheet
    data = [[
        q['id'],
        q['question'],
        q['option_a'],
        q['option_b'],
        q['option_c'],
        q['option_d'],
        q['correct_answer']
    ] for q in structured_questions]

    # Update worksheet
    new_worksheet.update(f'A2:G{len(data)+1}', data)

    # Verify answers were properly integrated
    questions_with_answers = sum(1 for q in structured_questions if q['correct_answer'])
    print(f"\nQuestions with answers: {questions_with_answers}/{len(structured_questions)}")

    print("\nSample of first parsed question with answer:")
    print(json.dumps(structured_questions[0], ensure_ascii=False, indent=2))

    print("\nFiles saved:")
    print("1. structured_questions.csv")
    print("2. structured_questions.json")
    print("3. New worksheet 'Structured_Questions' in your Google Sheet")

except Exception as e:
    print(f"Error: {str(e)}")

Authenticating...
Getting data from sheets...
Getting answers from Answers Matrix...
Parsing questions and integrating answers...
Successfully parsed 174 questions
Saving to CSV...
Saving to JSON...
Creating new worksheet...


<ipython-input-3-41567423ff60>:129: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  new_worksheet.update('A1:G1', [headers])
<ipython-input-3-41567423ff60>:143: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  new_worksheet.update(f'A2:G{len(data)+1}', data)



Questions with answers: 174/174

Sample of first parsed question with answer:
{
  "id": "26",
  "question": "En relación con el metabolismo del hierro y su control mediado por hepcidina, es cierto que:",
  "option_a": "La caída en la presión parcial de oxígeno promueve la activación del factor inducible por hipoxia (HIF), que aumenta la expresión de hepcidina.",
  "option_b": "El aumento de hierro sérico o la inflamación estimulan la síntesis de hepcidina en el hígado, que regula negativamente la función de la ferroportina.",
  "option_c": "La hepcidina disminuye la absorción intestinal de hierro a través de la inactivación del transportador de metales bivalentes 1 (DMT1).",
  "option_d": "En la hemocromatosis hereditaria de tipo 1, las mutaciones en la proteína de la hemocromatosis humana (HFE) causan un aumento de la producción de hepcidina.",
  "correct_answer": "B"
}

Files saved:
1. structured_questions.csv
2. structured_questions.json
3. New worksheet 'Structured_Questions' in y

In [ ]:
from google.colab import auth
import google.auth
import gspread
import pandas as pd
import json

def parse_questions(raw_data):
    questions = []
    current_question = None
    current_options = []

    # Skip header row
    for row in raw_data[1:]:
        text = row[0].strip()

        # Check if this is a question (starts with number followed by dot)
        question_match = re.match(r'^(\d{2,3})\.\s*(.*)', text)  # Matches 26. or 210.
        option_match = re.match(r'^([1-4])\.\s*(.*)', text)      # Matches 1. to 4.

        if question_match:
            # Save previous question if exists
            if current_question and len(current_options) == 4:
                questions.append({
                    'id': current_question['id'],
                    'question': current_question['text'],
                    'option_a': current_options[0],
                    'option_b': current_options[1],
                    'option_c': current_options[2],
                    'option_d': current_options[3],
                    'correct_answer': ''  # To be filled later
                })

            # Start new question
            current_question = {
                'id': question_match.group(1),
                'text': question_match.group(2)
            }
            current_options = []

        elif option_match and current_question:
            current_options.append(option_match.group(2))

    # Don't forget to add the last question
    if current_question and len(current_options) == 4:
        questions.append({
            'id': current_question['id'],
            'question': current_question['text'],
            'option_a': current_options[0],
            'option_b': current_options[1],
            'option_c': current_options[2],
            'option_d': current_options[3],
            'correct_answer': ''
        })

    return questions

try:
    # Authenticate
    print("Authenticating...")
    auth.authenticate_user()
    creds, _ = google.auth.default()
    gc = gspread.authorize(creds)

    # Your Google Sheet URL
    sheet_url = 'https://docs.google.com/spreadsheets/d/1rpqI5GlJUBmoa6gtrksgH9KrEaBXpD5xzCSgd_BH2Fw/edit?usp=sharing'

    # Open spreadsheet and get raw data
    print("Getting data from sheet...")
    spreadsheet = gc.open_by_url(sheet_url)
    worksheet = spreadsheet.worksheet('Questions2024')
    raw_data = worksheet.get_all_values()

    # Parse questions
    print("Parsing questions...")
    structured_questions = parse_questions(raw_data)

    print(f"Successfully parsed {len(structured_questions)} questions")

    # Save as CSV
    print("Saving to CSV...")
    df = pd.DataFrame(structured_questions)
    df.to_csv('structured_questions.csv', index=False)

    # Save as JSON
    print("Saving to JSON...")
    with open('structured_questions.json', 'w', encoding='utf-8') as f:
        json.dump(structured_questions, f, ensure_ascii=False, indent=2)

    # Create new worksheet in Google Sheet
    print("Creating new worksheet...")
    try:
        new_worksheet = spreadsheet.add_worksheet('Structured_Questions', 1000, 7)
    except:
        new_worksheet = spreadsheet.worksheet('Structured_Questions')
        new_worksheet.clear()

    # Add headers
    headers = ['QuestionID', 'Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Correct Answer']
    new_worksheet.update('A1:G1', [headers])

    # Prepare data for worksheet
    data = [[
        q['id'],
        q['question'],
        q['option_a'],
        q['option_b'],
        q['option_c'],
        q['option_d'],
        q['correct_answer']
    ] for q in structured_questions]

    # Update worksheet
    new_worksheet.update(f'A2:G{len(data)+1}', data)

    print("\nSample of first parsed question:")
    print(json.dumps(structured_questions[0], ensure_ascii=False, indent=2))

    print("\nFiles saved:")
    print("1. structured_questions.csv")
    print("2. structured_questions.json")
    print("3. New worksheet 'Structured_Questions' in your Google Sheet")

except Exception as e:
    print(f"Error: {str(e)}")

Authenticating...
Getting data from sheet...
Parsing questions...
Successfully parsed 174 questions
Saving to CSV...
Saving to JSON...
Creating new worksheet...


<ipython-input-2-74c2d03d07e8>:99: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  new_worksheet.update('A1:G1', [headers])
<ipython-input-2-74c2d03d07e8>:113: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  new_worksheet.update(f'A2:G{len(data)+1}', data)



Sample of first parsed question:
{
  "id": "26",
  "question": "En relación con el metabolismo del hierro y su control mediado por hepcidina, es cierto que:",
  "option_a": "La caída en la presión parcial de oxígeno promueve la activación del factor inducible por hipoxia (HIF), que aumenta la expresión de hepcidina.",
  "option_b": "El aumento de hierro sérico o la inflamación estimulan la síntesis de hepcidina en el hígado, que regula negativamente la función de la ferroportina.",
  "option_c": "La hepcidina disminuye la absorción intestinal de hierro a través de la inactivación del transportador de metales bivalentes 1 (DMT1).",
  "option_d": "En la hemocromatosis hereditaria de tipo 1, las mutaciones en la proteína de la hemocromatosis humana (HFE) causan un aumento de la producción de hepcidina.",
  "correct_answer": ""
}

Files saved:
1. structured_questions.csv
2. structured_questions.json
3. New worksheet 'Structured_Questions' in your Google Sheet
